# Variational Autoencoders

In [1]:
#Imports 
import sys
sys.path.append("../../") 
from sklearn.metrics import confusion_matrix, classification_report
from model.predict import *
from model.helpers import *
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load data 
TRAIN = {}
TEST = {}
VALIDATION = {}

path = "../../../data/interim/raw"
train = "train_"
test = "test_"
validation = "validation_"
#  list of scenarios
scenarios = [
        'CVE-2012-2122',
        'CVE-2014-0160',
        'CVE-2017-7529',
        'CVE-2017-12635_6',
        'CVE-2018-3760',
        'CVE-2019-5418',
        'CVE-2020-9484',
        'CVE-2020-13942',
        'CVE-2020-23839',
        'CWE-89-SQL-injection'
]

for i in range(0,len(scenarios)):
    TRAIN[scenarios[i]]= pd.read_pickle(path + "/" + train + scenarios[i] + ".pkl")
    TEST[scenarios[i]] = pd.read_pickle(path + "/" + test + scenarios[i] + ".pkl")
    VALIDATION[scenarios[i]] = pd.read_pickle(path + "/" + validation + scenarios[i] + ".pkl")
            

In [3]:
load_model(model_type='VariationalAutoencoder', data=TRAIN, scenario='CVE-2012-2122')

ValueError: The number of neurons should not exceed the number of features

In [4]:
models = {}
for i in range(0,len(scenarios)):
    models[scenarios[i]] = load_model(model_type='VariationalAutoencoder', data=TRAIN, scenario=scenarios[i])

ValueError: The number of neurons should not exceed the number of features